El objetivo de esta notebook es poner en pratica lo aprendido en Web Scrapping con Bs4 para obtener una lita con las notebooks ofrecidas en la primera page de Amazon.

In [1]:
import bs4
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import pandas as pd

In [2]:
amazon_url = 'https://www.amazon.es/s?k=notebook+i5+ssd+4+gb&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss'

In [3]:
uClient = uReq(amazon_url) #abre la url y la descarga
page_html = uClient.read() #leo los datos en una var
uClient.close() #cierro la conexion

In [4]:
page_soup = soup(page_html, "html.parser") #html parsing

In [5]:
page_soup.p #exploro el bloque html! encontre una p type, puedo buscar divs, a, span, lo que se me ocurra

<p class="a-spacing-none">Utilizamos cookies y herramientas similares para mejorar tu experiencia de compra, prestar nuestros servicios, entender cómo los utilizas para poder mejorarlos, y para mostrarte anuncios. Terceros autorizados también utilizan estas herramientas en relación con los anuncios que mostramos.</p>

In [6]:
#agarro cada producto ahora.. esa clase es del titulo
pcs = page_soup.findAll("div", {"class":"sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32"})
len(pcs) #encontro 48 notebooks

48

In [7]:
pcs[0] #vemos que es una lista con la info de cada una
       #inspceccionamos un poco

<div class="sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32" data-asin="B07Y8R8SG9" data-component-type="s-search-result" data-index="0" data-uuid="c6d23707-8b79-46ae-9ef6-f1f38646b01a"><div class="sg-col-inner">
<span cel_widget_id="MAIN-SEARCH_RESULTS-0" class="celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results">
<div class="s-expand-height s-include-content-margin s-border-bottom s-latency-cf-section">
<div class="a-section a-spacing-medium">
<div class="a-section a-spacing-micro s-grid-status-badge-container">
</div>
<span class="rush-component" data-component-type="s-product-image">
<a class="a-link-normal s-no-outline" href="/Microsoft-Surface-Pro-Ordenador-i5-1035G4/dp/B07Y8R8SG9?dchild=1">
<div class="a-section aok-relative s-image-square-aspect">
<img alt='Microsoft Surface Pro 7 - Ordenador portátil 2 en 1 de 12.3" (Intel Core i5-1035G4, 8GB RAM, 256GB SSD, Intel Graphics, Wi

In [8]:
#ahora voy a meterme en el html de cada publicacion para obtener la descripcion y el precio

filename = 'notebooks_amazon.csv'
f = open(filename, 'w')

headers = 'descripcion, precio \n'
f.write(headers)

for pc in pcs:
    descripcion_html = pc.findAll('span', {'class':'a-size-base-plus a-color-base a-text-normal'})
    descripcion = descripcion_html[0].text
    precio_html = pc.findAll('span', {'class':'a-price-whole'})
    if len(precio_html) >= 1:
        precio = precio_html[0].text
    else:
        precio = 0
    rating_html = pc.findAll('span', {"aria-label": '4,3 de 5 estrellas'})
    if len(rating_html) >= 1:
        rating = rating_html[0].text.strip()
    else:
        rating = 0   
    
    #envio = pc.findAll('span', {'aria-label':"Entrega GRATIS"})
    
    print('Descripción: ' + descripcion)
    print('Precio: ' + str(precio))
    print('Rating: ' + str(rating))
    
    f.write(descripcion.replace(',','/') + ',' + str(precio).replace(',','.') + str(rating) + '\n')
    
f.close()    

Descripción: Microsoft Surface Pro 7 - Ordenador portátil 2 en 1 de 12.3" (Intel Core i5-1035G4, 8GB RAM, 256GB SSD, Intel Graphics, Windows 10) Negro
Precio: 944,00
Rating: 0
Descripción: Medion ERAZER P6605 - Portátil gaming 15.6" FullHD (Intel Core i5-8300H, 16GB RAM, 256GB SSD, Nvidia GeForce GTX 1050-4GB, Windows 10) - Teclado QWERTY Español
Precio: 699,99
Rating: 0
Descripción: ASUS TUF Gaming A15 FX506LH-BQ034 - Ordenador Portátil Gaming de 15.6" FullHD (Intel Core i5-10300H, 16GB RAM, 512GB SSD, NVIDIA GTX1650-4GB, Sin Sistema Operativo)Negro Hoguera-Teclado QWERTY español
Precio: 899,99
Rating: 4,3 de 5 estrellas
Descripción: HP 15s-fq1089ns - Ordenador portátil de 15.6" FullHD (Intel Core i5-1035G1, 8GB RAM, 512GB SSD, Intel UHD Graphics, Windows 10 Home) plata - Teclado QWERTY Español
Precio: 649,99
Rating: 0
Descripción: HP Pavilion 15-bc521ns - Ordenador portátil de 15.6" FullHD (Intel Core i5-9300H, 16GB RAM, 512GB SSD, NVIDIA GTX 1650-4GB, Sin sistema operativo), Color N

In [9]:
#lo hago con pandas ahora 

descripciones = []
precios = []
ratings = []

for pc in pcs:
    descripcion_html = pc.findAll('span', {'class':'a-size-base-plus a-color-base a-text-normal'})
    descripcion = descripcion_html[0].text
    precio_html = pc.findAll('span', {'class':'a-price-whole'})
    if len(precio_html) >= 1: #si findAll no encuentra un dato te tira una lista vacia! y algnos precios faltan..
        precio = precio_html[0].text
    else:
        precio = 0
    rating_html = pc.findAll('span', {"aria-label": '4,3 de 5 estrellas'})
    if len(rating_html) >= 1:
        rating = rating_html[0].text.strip()
    else:
        rating = 0   
    
    #envio = pc.findAll('span', {'aria-label':"Entrega GRATIS"})
    
    print('Descripción: ' + descripcion)
    print('Precio: ' + str(precio))
    print('Rating: ' + str(rating))
    
    descripciones.append(descripcion)
    precios.append(precio)
    ratings.append(rating)
    
data = pd.DataFrame([descripciones, precios, ratings])

Descripción: Microsoft Surface Pro 7 - Ordenador portátil 2 en 1 de 12.3" (Intel Core i5-1035G4, 8GB RAM, 256GB SSD, Intel Graphics, Windows 10) Negro
Precio: 944,00
Rating: 0
Descripción: Medion ERAZER P6605 - Portátil gaming 15.6" FullHD (Intel Core i5-8300H, 16GB RAM, 256GB SSD, Nvidia GeForce GTX 1050-4GB, Windows 10) - Teclado QWERTY Español
Precio: 699,99
Rating: 0
Descripción: ASUS TUF Gaming A15 FX506LH-BQ034 - Ordenador Portátil Gaming de 15.6" FullHD (Intel Core i5-10300H, 16GB RAM, 512GB SSD, NVIDIA GTX1650-4GB, Sin Sistema Operativo)Negro Hoguera-Teclado QWERTY español
Precio: 899,99
Rating: 4,3 de 5 estrellas
Descripción: HP 15s-fq1089ns - Ordenador portátil de 15.6" FullHD (Intel Core i5-1035G1, 8GB RAM, 512GB SSD, Intel UHD Graphics, Windows 10 Home) plata - Teclado QWERTY Español
Precio: 649,99
Rating: 0
Descripción: HP Pavilion 15-bc521ns - Ordenador portátil de 15.6" FullHD (Intel Core i5-9300H, 16GB RAM, 512GB SSD, NVIDIA GTX 1650-4GB, Sin sistema operativo), Color N

In [10]:
data = data.transpose()
data.columns = ['descripcion', 'precio', 'rating']
data.head(5)

,descripcion,precio,rating
0,Microsoft Surface Pro 7 - Ordenador portátil 2...,"944,00",0
1,"Medion ERAZER P6605 - Portátil gaming 15.6"" Fu...","699,99",0
2,ASUS TUF Gaming A15 FX506LH-BQ034 - Ordenador ...,"899,99","4,3 de 5 estrellas"
3,"HP 15s-fq1089ns - Ordenador portátil de 15.6"" ...","649,99",0
4,HP Pavilion 15-bc521ns - Ordenador portátil de...,0,0
